In [1]:
import yaml
import pandas as pd
import numpy as np
from pathlib import Path
import os

# ==============================================================================
# 1. CARGA DE CONFIGURAÇÃO
# ==============================================================================

def find_project_root(anchor_file="conf/config.yaml"):
    """
    Sobe os diretórios a partir do notebook atual até encontrar
    a pasta onde 'conf/config.yaml' existe.
    """
    current_path = Path.cwd()
    
    # Tenta no diretório atual e sobe até o raiz do sistema
    for parent in [current_path] + list(current_path.parents):
        potential_config = parent / anchor_file
        if potential_config.exists():
            return parent
            
    raise FileNotFoundError(f"Não foi possível encontrar a raiz do projeto contendo '{anchor_file}'.")

# 1. Definir BASE_DIR (Raiz do Projeto)
try:
    BASE_DIR = find_project_root("conf/config.yaml")
    print(f"📂 Raiz do Projeto encontrada: {BASE_DIR}")
except FileNotFoundError as e:
    # Fallback manual caso a busca automática falhe (ajuste se necessário)
    print("Busca automática falhou. Usando fallback.")
    BASE_DIR = Path("/Users/lucasborges/Downloads/TCC")

# 2. Carregar o YAML da pasta conf
CONFIG_PATH = BASE_DIR / "conf/config.yaml"
with open(CONFIG_PATH, "r") as f:
    config = yaml.safe_load(f)

# ==============================================================================
# 2. ATALHOS E VARIÁVEIS GLOBAIS
# ==============================================================================

# Atalhos dos Dicionários do YAML
# P['raw'] vai virar algo como: /Users/.../TCC/data/raw
P = {k: BASE_DIR / v for k, v in config['paths'].items()} # P de Paths
F = config['files']                                       # F de Files
PM = config['params']                                     # PM de Params

print(f"⚙️ Configuração carregada de: {CONFIG_PATH}")

# ==============================================================================
# 3. PONTE DE VARIÁVEIS
# ==============================================================================

# Caminhos de Arquivos (Apontando para o YAML)
TRAIN_EMB_PATH       = P['processed'] / F['track_embeddings']
NEW_EMB_PATH         = P['processed'] / F['new_track_embeddings']
X_TRAIN_PATH         = P['processed'] / F['train_features']
X_TEST_PATH          = P['processed'] / F['test_features']


# Se não estiver no YAML, usa o caminho construído:
TRACKS_COMPLETE_PATH = P['interim']   / "df_tracks_complete_v5.parquet"

# Caminhos de Grafos
# Verifica se as chaves existem no yaml, senão usa padrão
MATCHING_MAP_PATH    = P.get('graphs_coarsened', P['graphs_bipartite']) / F['matching_map']
SUPER_EMB_PATH       = P.get('graphs_super', P['graphs_bipartite'])     / F['super_embeddings']

# Parâmetros
SEED                 = PM['seed']

# Configurações Visuais Padrão
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_style("whitegrid")
plt.rcParams['figure.dpi'] = 300

📂 Raiz do Projeto encontrada: /Users/lucasborges/Downloads/TCC
⚙️ Configuração carregada de: /Users/lucasborges/Downloads/TCC/conf/config.yaml


In [2]:
# Dados Raw
df_playlists = pd.read_parquet(P['raw'] / "df_playlists_v0.parquet")
df_tracks    = pd.read_parquet(P['raw'] / "df_tracks_v0.parquet")
dim_artists  = pd.read_parquet(P['raw'] / "dim_artists.parquet")
dim_albums   = pd.read_parquet(P['raw'] / "dim_albums.parquet")

# Dados Processados 
df_full      = pd.read_parquet(P['processed'] / "df_full.parquet")
X_train      = pd.read_parquet(P['processed'] / F['train_features'])
X_test       = pd.read_parquet(P['processed'] / F['test_features'])

print("Todos os arquivos carregados com sucesso!")

Todos os arquivos carregados com sucesso!


In [3]:
import pandas as pd
import networkx as nx
from networkx.algorithms import bipartite
import numpy as np
from collections import defaultdict
from itertools import combinations

In [4]:
# ==============================================================================
# ETAPA 1: Filtrar tracks válidas (presentes em X_train_v1)
# ==============================================================================
valid_tracks = set(X_train['track_uri'].unique())
print(f"Total de tracks válidas: {len(valid_tracks):,}")

# ==============================================================================
# ETAPA 2: Filtrar df_tracks para manter apenas tracks válidas
# ==============================================================================
df_tracks_filtered = df_tracks[df_tracks['track_uri'].isin(valid_tracks)].copy()
print(f"Registros originais em df_tracks: {len(df_tracks):,}")
print(f"Registros após filtro: {len(df_tracks_filtered):,}")

Total de tracks válidas: 324,305
Registros originais em df_tracks: 6,685,101
Registros após filtro: 2,821,538


In [5]:
from scipy.sparse import coo_matrix

def build_bipartite_from_dfs(df_playlists, df_tracks,
                             alpha=1.0, beta=0.3, gamma=0.5,
                             pos_lambda=3.0, half_life_days=90):
    # Merge para trazer num_tracks e modified_at
    df = df_tracks.merge(
        df_playlists[['pid','num_tracks','modified_at']],
        on='pid', how='left'
    )

    # --- posWeight ---
    L = df['num_tracks'].clip(lower=1)
    pos_norm = np.where(L>1, df['pos']/(L-1), 0.0)
    pos_weight = np.exp(-pos_lambda * pos_norm)

    # --- timeWeight ---
    now = pd.Timestamp.utcnow().timestamp()
    delta_days = (now - df['modified_at']) / 86400.0  # segundos -> dias
    time_weight = np.exp(-np.log(2) * (delta_days / half_life_days))

    # --- peso final ---
    w = alpha*1.0 + beta*pos_weight + gamma*time_weight
    w = w / np.sqrt(L)  # normalizar por tamanho da playlist

    # --- Mapas p→linha, m→coluna ---
    p_index = pd.Index(df['pid'].unique(), name='playlist_id')
    m_index = pd.Index(df['track_uri'].unique(), name='track_id')

    p_codes = df['pid'].map({pid:i for i,pid in enumerate(p_index)}).to_numpy()
    m_codes = df['track_uri'].map({tid:i for i,tid in enumerate(m_index)}).to_numpy()

    # Matriz esparsa Playlist × Música
    B = coo_matrix((w, (p_codes, m_codes)),
                   shape=(len(p_index), len(m_index))).tocsr()

    return B, p_index, m_index


In [6]:
# ==============================================================================
# ETAPA 3: Criar a matriz bipartida B (playlists x tracks)
# ==============================================================================

print("\nCriando matriz bipartida B com build_bipartite_from_dfs...")

B, p_index, m_index = build_bipartite_from_dfs(
    df_playlists=df_playlists,
    df_tracks=df_tracks_filtered,
    alpha=1.0,
    beta=0.3,
    gamma=0.5,
    pos_lambda=3.0,
    half_life_days=90
)

print("Matriz B criada!")
print(f"Shape de B: {B.shape}  (playlists x tracks)")
print(f"Número de entradas não nulas (arestas): {B.nnz:,}")

# ==============================================================================
# ETAPA 4: Validar a rede bipartida a partir de B
# ==============================================================================

print("\n" + "="*70)
print("INFORMAÇÕES DA REDE BIPARTIDA (a partir da matriz B)")
print("="*70)

nP, nM = B.shape
nnz = B.nnz

# 1) É bipartida?
# Por construção, sim: linhas = playlists, colunas = tracks.
is_bipartite = True
print(f"É bipartida: {is_bipartite}")

# 2) Estatísticas gerais
print(f"\nTotal de nós: {nP + nM:,}  (playlists={nP:,}, tracks={nM:,})")
print(f"Total de arestas (pares playlist-track com w_pm > 0): {nnz:,}")
density = nnz / (nP * nM) if nP * nM > 0 else 0.0
print(f"Densidade (nnz / (|P| * |M|)): {density:.6e}")

# 3) "Conjuntos" de nós (só para manter o paralelo com o código antigo)
print(f"\nNós do tipo playlist: {nP:,}")
print(f"Nós do tipo track: {nM:,}")

# 4) Estatísticas de grau (em B, é número de entradas !=0 por linha/coluna)
#    Playlist: quantas músicas ela contém
#    Track: em quantas playlists aparece

# CSR: indptr guarda offsets das linhas => diff = nnz por linha
playlist_degrees = np.diff(B.indptr)           # nº de tracks por playlist
# Colunas: usa CSC, mesmo raciocínio
Bc = B.tocsc()
track_degrees = np.diff(Bc.indptr)             # nº de playlists por track

print(f"\n--- Grau das Playlists (número de músicas por playlist) ---")
print(f"Média:   {np.mean(playlist_degrees):.2f}")
print(f"Mediana: {np.median(playlist_degrees):.2f}")
print(f"Mínimo:  {np.min(playlist_degrees)}")
print(f"Máximo:  {np.max(playlist_degrees)}")

print(f"\n--- Grau das Tracks (número de playlists que contêm cada música) ---")
print(f"Média:   {np.mean(track_degrees):.2f}")
print(f"Mediana: {np.median(track_degrees):.2f}")
print(f"Mínimo:  {np.min(track_degrees)}")
print(f"Máximo:  {np.max(track_degrees)}")

# 5) (Opcional) Estatísticas dos pesos w_pm
w = B.data
print(f"\n--- Pesos das arestas w_pm ---")
print(f"Peso médio: {w.mean():.4f}")
print(f"Peso mínimo: {w.min():.4f}")
print(f"Peso máximo: {w.max():.4f}")
print("Percentis [1,25,50,75,90,99]:", np.round(np.percentile(w, [1,25,50,75,90,99]), 4))


Criando matriz bipartida B com build_bipartite_from_dfs...
Matriz B criada!
Shape de B: (97948, 324305)  (playlists x tracks)
Número de entradas não nulas (arestas): 2,782,022

INFORMAÇÕES DA REDE BIPARTIDA (a partir da matriz B)
É bipartida: True

Total de nós: 422,253  (playlists=97,948, tracks=324,305)
Total de arestas (pares playlist-track com w_pm > 0): 2,782,022
Densidade (nnz / (|P| * |M|)): 8.758129e-05

Nós do tipo playlist: 97,948
Nós do tipo track: 324,305

--- Grau das Playlists (número de músicas por playlist) ---
Média:   28.40
Mediana: 19.00
Mínimo:  1
Máximo:  224

--- Grau das Tracks (número de playlists que contêm cada música) ---
Média:   8.58
Mediana: 1.00
Mínimo:  1
Máximo:  3105

--- Pesos das arestas w_pm ---
Peso médio: 0.1247
Peso mínimo: 0.0642
Peso máximo: 2.4453
Percentis [1,25,50,75,90,99]: [0.0668 0.0872 0.1095 0.1456 0.1925 0.3177]


In [7]:
from scipy.sparse import bmat
from scipy.sparse.csgraph import connected_components

# ==============================================================================
# ETAPA 6: Análise de Componentes Conectados (via matriz B)
# ==============================================================================

print("\n" + "="*70)
print("ANÁLISE DE COMPONENTES CONECTADOS")
print("="*70)

# B: matriz bipartida (playlists x tracks) em CSR
nP, nM = B.shape
print(f"Tamanho de B: {nP:,} playlists x {nM:,} tracks")

# Monta matriz de adjacência do grafo bipartido:
# A = [[0, B],
#      [B.T, 0]]
A = bmat([[None, B],
          [B.T, None]], format="csr")

# Obter componentes conectados
n_components, labels = connected_components(A, directed=False, return_labels=True)

print(f"\nNúmero total de componentes conectados: {n_components:,}")

# labels: tamanho nP + nM
#  - primeiros nP índices correspondem às playlists (linhas de B)
#  - últimos nM índices correspondem às tracks (colunas de B)
labels_P = labels[:nP]
labels_M = labels[nP:]

# Tamanho de cada componente
# comp_id -> tamanho
component_ids, component_sizes = np.unique(labels, return_counts=True)

# Ordenar tamanhos (decrescente)
component_sizes_sorted = sorted(component_sizes.tolist(), reverse=True)

print(f"\nDistribuição de tamanhos das componentes:")
print(f"- Maior componente (LCC): {component_sizes_sorted[0]:,} nós")
if len(component_sizes_sorted) > 1:
    print(f"- Segunda maior: {component_sizes_sorted[1]:,} nós")
if len(component_sizes_sorted) > 2:
    print(f"- Terceira maior: {component_sizes_sorted[2]:,} nós")

# Percentual da LCC
total_nodes = nP + nM
lcc_size = component_sizes_sorted[0]
lcc_percentage = (lcc_size / total_nodes) * 100 if total_nodes > 0 else 0.0
print(f"\nLCC representa {lcc_percentage:.2f}% dos nós totais")

# Identificar ID da LCC
largest_comp_id = component_ids[np.argmax(component_sizes)]

# Índices (posições) dos nós na LCC
playlists_in_lcc_idx = np.where(labels_P == largest_comp_id)[0]
tracks_in_lcc_idx    = np.where(labels_M == largest_comp_id)[0]

print(f"\nNós na LCC: {lcc_size:,}")
print(f"- Playlists na LCC: {len(playlists_in_lcc_idx):,}")
print(f"- Tracks na LCC: {len(tracks_in_lcc_idx):,}")

# Se quiser os IDs reais (pid / track_uri), pode mapear assim:
# playlists_in_lcc_ids = p_index[playlists_in_lcc_idx]
# tracks_in_lcc_ids    = m_index[tracks_in_lcc_idx]

# Estatísticas das componentes pequenas
if n_components > 1:
    small_components = component_sizes_sorted[1:]
    print(f"\nComponentes isoladas/pequenas:")
    print(f"- Quantidade: {len(small_components):,}")
    print(f"- Tamanho médio: {np.mean(small_components):.2f} nós")
    print(f"- Tamanho mínimo: {min(small_components)} nós")
    print(f"- Tamanho máximo: {max(small_components)} nós")



ANÁLISE DE COMPONENTES CONECTADOS
Tamanho de B: 97,948 playlists x 324,305 tracks

Número total de componentes conectados: 353

Distribuição de tamanhos das componentes:
- Maior componente (LCC): 418,754 nós
- Segunda maior: 147 nós
- Terceira maior: 117 nós

LCC representa 99.17% dos nós totais

Nós na LCC: 418,754
- Playlists na LCC: 97,594
- Tracks na LCC: 321,160

Componentes isoladas/pequenas:
- Quantidade: 352
- Tamanho médio: 9.94 nós
- Tamanho mínimo: 2 nós
- Tamanho máximo: 147 nós


In [8]:
# ==============================================================================
# ETAPA 7: Criar matriz bipartida apenas com a LCC
# ==============================================================================

print("\n" + "="*70)
print("CRIANDO MATRIZ B APENAS COM A LCC")
print("="*70)

# 1) Recortar B para manter apenas playlists e tracks da LCC
B_lcc = B[playlists_in_lcc_idx[:, None], tracks_in_lcc_idx]  # forma 1
# ou, de forma equivalente e mais legível:
B_lcc = B[playlists_in_lcc_idx][:, tracks_in_lcc_idx]

# 2) Índices (ids) correspondentes
p_index_lcc = p_index[playlists_in_lcc_idx]  # pids na LCC
m_index_lcc = m_index[tracks_in_lcc_idx]     # track_uris na LCC

print(f"\nMatriz B_lcc criada!")
nP_lcc, nM_lcc = B_lcc.shape
nnz_lcc = B_lcc.nnz

print(f"Playlists na LCC: {nP_lcc:,}")
print(f"Tracks na LCC: {nM_lcc:,}")
print(f"Total de nós na LCC: {nP_lcc + nM_lcc:,}")
print(f"Total de arestas na LCC: {nnz_lcc:,}")

density_lcc = nnz_lcc / (nP_lcc * nM_lcc) if nP_lcc * nM_lcc > 0 else 0.0
print(f"Densidade da LCC: {density_lcc:.6e}")

# 3) Estatísticas de grau na LCC
#    Playlist: nº de músicas; Track: nº de playlists

# CSR: indptr => graus por linha (playlists)
playlist_degrees_lcc = np.diff(B_lcc.indptr)

# Colunas: converter para CSC para obter graus por coluna (tracks)
B_lcc_csc = B_lcc.tocsc()
track_degrees_lcc = np.diff(B_lcc_csc.indptr)

print(f"\n--- Grau das Playlists na LCC (nº de músicas por playlist) ---")
print(f"Média:   {np.mean(playlist_degrees_lcc):.2f}")
print(f"Mediana: {np.median(playlist_degrees_lcc):.2f}")
print(f"Mínimo:  {np.min(playlist_degrees_lcc)}")
print(f"Máximo:  {np.max(playlist_degrees_lcc)}")

print(f"\n--- Grau das Tracks na LCC (nº de playlists por track) ---")
print(f"Média:   {np.mean(track_degrees_lcc):.2f}")
print(f"Mediana: {np.median(track_degrees_lcc):.2f}")
print(f"Mínimo:  {np.min(track_degrees_lcc)}")
print(f"Máximo:  {np.max(track_degrees_lcc)}")

print("\n" + "="*70)
print("Redes/matrizes criadas com sucesso!")
print("- B: Matriz bipartida completa (playlists x tracks)")
print("- B_lcc: Matriz apenas com a LCC (playlists_lcc x tracks_lcc)")
print("="*70)



CRIANDO MATRIZ B APENAS COM A LCC

Matriz B_lcc criada!
Playlists na LCC: 97,594
Tracks na LCC: 321,160
Total de nós na LCC: 418,754
Total de arestas na LCC: 2,778,857
Densidade da LCC: 8.865876e-05

--- Grau das Playlists na LCC (nº de músicas por playlist) ---
Média:   28.47
Mediana: 19.00
Mínimo:  1
Máximo:  224

--- Grau das Tracks na LCC (nº de playlists por track) ---
Média:   8.65
Mediana: 1.00
Mínimo:  1
Máximo:  3105

Redes/matrizes criadas com sucesso!
- B: Matriz bipartida completa (playlists x tracks)
- B_lcc: Matriz apenas com a LCC (playlists_lcc x tracks_lcc)


In [9]:
from scipy.sparse import save_npz

# Definir pasta base para grafos bipartidos
base_path = P['graphs_bipartite']

print(f"Salvando grafos em: {base_path}")

# 1. Salvar Matrizes Esparsas
save_npz(base_path / "B_full.npz", B)
save_npz(base_path / "B_lcc.npz", B_lcc)

# 2. Salvar Índices (Series -> DataFrame -> Parquet)
# Full Graph Indices
p_index.to_frame(name="pid").to_parquet(base_path / "p_index.parquet")
m_index.to_frame(name="track_uri").to_parquet(base_path / "m_index.parquet")

# Largest Connected Component (LCC) Indices
p_index_lcc.to_frame(name="pid").to_parquet(base_path / "p_index_lcc.parquet")
m_index_lcc.to_frame(name="track_uri").to_parquet(base_path / "m_index_lcc.parquet")

print("Arquivos bipartidos salvos com sucesso!")

Salvando grafos em: /Users/lucasborges/Downloads/TCC/graphs/bipartite
Arquivos bipartidos salvos com sucesso!
